# Script 1:

This script uses the P- and S-wave picks from a catalogued earthquake to retrieve the waveforms from the continuous data. Then, it creates a template event usable in a subsequent matched filter search. This script can be generalized to generate a database of template events using a whole earthquake catalog.

In [ ]:
import sys
import os
sys.path.append(os.getcwd())

import h5py as h5
import numpy as np
import utils

from obspy.core import UTCDateTime as udt
import matplotlib.pyplot as plt

## Load the continuous seismic data and the event metadata

Our data are stored in an h5 file, and we load them through the *load_data* function that returns a Python dictionary containing the waveforms and the relevant metadata. Similarly, the template metadata are stored in an h5 file and loaded in a Python dictionary.

In [ ]:
# load the data from day 2013-03-17
data = utils.load_data('data_FMF_tutorial.h5')
print('Elements in data:\n', data.keys())
print('metadata is also a Python dictionary:\n', data['metadata'].keys())
print('\n')

# load the metadata from a template event detected on day 2013-03-17
template_metadata = utils.load_template('template_FMF_tutorial.h5')
print('Elements in template_metadata:\n', template_metadata.keys())

## Retrieve the P- and S-wave picks from the origin time and the travel times

In [ ]:
# use the metadata to extract the waveforms of the template event
# first, retrieve the origin time
origin_time = template_metadata['origin_time']
# this time is a timestamp in seconds
print('Timestamp in seconds: {:.2f}, human readable date: {}'.\
        format(origin_time, udt(origin_time).strftime('%Y,%m,%d--%H:%M:%S')))

# second, using the travel times we build the picks
# for both the P and S waves
tt_P = template_metadata['p_travel_times']
tt_S = template_metadata['s_travel_times']

# the picks, or the phase arrivals, are the sum
# of the origin time and the travel times
picks_P = origin_time + np.float64(tt_P)
picks_S = origin_time + np.float64(tt_S)
for s in range(len(template_metadata['stations'])):
    print('P pick on station {}: {}'.format(template_metadata['stations'][s].decode('utf-8'),
                                            udt(picks_P[s]).strftime('%Y,%m,%d--%H:%M:%S')))
    print('S pick on station {}: {}'.format(template_metadata['stations'][s].decode('utf-8'),
                                            udt(picks_S[s]).strftime('%Y,%m,%d--%H:%M:%S')))
    print('\n')

## Extract the time windows of interest
**IMPORTANT: BEFORE EXTRACTING ANY WAVEFORMS, WE HAVE TO CONVERT ALL THE TIMES IN SECONDS TO TIMES IN SAMPLES**<br/>
The reason for that is that the template moveouts are ultimately transformed to times in samples. To avoid rounding errors resulting from inconsistencies between the moveouts used when extracting the template waveforms and the moveouts converted to samples, we convert all times to samples before extracting the waveforms.

In [ ]:
# get the timestamp of the the beginning of your data
# in this example, the data start at 2013,03,17 00:00:00
T0 = udt('2013,03,17').timestamp

# we can now define our picks as times relative to T0
# and expressed in number of samples
#
# get the sampling rate:
SR = data['metadata']['sampling_rate']

# use the sampling rate to convert the times
picks_P_samples = np.int32((picks_P - T0) * SR)
picks_S_samples = np.int32((picks_S - T0) * SR)

# let say we want to extract 1 second before the P wave
# and 4 seconds before the S wave
P_wave_buffer = np.int32(1. * SR)
S_wave_buffer = np.int32(4. * SR)

# these buffers define the beginning of the windows we want
# to extract on each station
beginning_P_windows = picks_P_samples - P_wave_buffer
beginning_S_windows = picks_S_samples - S_wave_buffer
print('We now have window start times in samples, relative to the beginning of the data array')
for s in range(len(template_metadata['stations'])):
    print('P window start time on station {}: {:d} samples'.format(
        template_metadata['stations'][s].decode('utf-8'),
        beginning_P_windows[s]))
    print('S window start time on station {}: {:d} samples'.format(
        template_metadata['stations'][s].decode('utf-8'),
        beginning_S_windows[s]))
    print('\n')

We are now ready to use the vectors **beginning_P_windows** and **beginning_S_windows** to extract the time windows from the continuous data. We choose a template length of 8 seconds.<br/>

Windows centered around the S wave are extracted on the horizontal components, and windows capturing the P wave are extracted on the vertical components.

In [ ]:
# extract the waveforms:
# let say we only want to extract the S wave on the
# horizontal components, and the P wave on the 
# vertical components
# we fix the template duration to 8 seconds
n_stations = template_metadata['stations'].size
n_components = len(data['metadata']['components'])
duration = np.int32(8. * SR)
template_waveforms = np.zeros((n_stations, n_components, duration),
                              dtype=np.float32)
for s in range(n_stations):
    for c in range(n_components):
        if c < 2:
            # the data are organized such that c=0 and c=1 are 
            # the indexes for components north/south and east/west
            idx_start = beginning_S_windows[s]
            idx_end = idx_start + duration
        else:
            idx_start = beginning_P_windows[s]
            idx_end = idx_start + duration
        template_waveforms[s, c, :] = data['waveforms'][s, c, idx_start:idx_end]

## Keep in memory the relative time shifts between each station / component, *i.e.* the moveouts

The moveouts are a collection of time shifts relative to a reference time: we choose this reference time to be the beginning of the earliest time window.

In [ ]:
# we now keep in memory the relative time shifts between each channel,
# which we call the moveouts
# the reference time is the earliest start time of the windows
reference_time = min(beginning_P_windows.min(), beginning_S_windows.min())
moveouts_P = beginning_P_windows - reference_time
moveouts_S = beginning_S_windows - reference_time

moveouts = np.hstack( (moveouts_S.reshape(-1, 1),
                       moveouts_S.reshape(-1, 1),
                       moveouts_P.reshape(-1, 1)) )
print('The horizontal and vertical component moveouts are:')
for s in range(len(template_metadata['stations'])):
    print('vertical component moveout on station {}: {:d} samples'.format(
        template_metadata['stations'][s].decode('utf-8'),
        moveouts_P[s]))
    print('horizontal component moveout on station {}: {:d} samples'.format(
        template_metadata['stations'][s].decode('utf-8'),
        moveouts_S[s]))
    print('\n')

## Save the template information in an h5 file

In [ ]:
# save the output in an h5 file
os.system('mkdir output')
with h5.File('./output/template.h5', mode='w') as f:
    with h5.File('./template_FMF_tutorial.h5', mode='r') as f2:
        for key in f2.keys():
            f.create_dataset(key, data=f2[key][()])
    f.create_dataset('moveouts_P', data=moveouts_P, compression='gzip')
    f.create_dataset('moveouts_S', data=moveouts_S, compression='gzip')
    f.create_dataset('sampling_rate', data=data['metadata']['sampling_rate'])
    f.create_dataset('waveforms', data=template_waveforms, compression='gzip')
    print('We have just created a h5 database featuring the following datasets:\n', list(f.keys()))

## Plot the template event

In [ ]:
# tune some plotting parameters
font = {'family': 'serif', 
        'size': 18}
plt.rc('font', **font)

In [ ]:
# plot the template event
mv_min = 0.
mv_max = max(moveouts_P.max(), moveouts_S.max())
time_min = mv_min
time_max = (duration + mv_max) / SR # in seconds
figsize = (28, 12) # note: you can play with figsize to better fit your monitor
plt.figure('template_event', figsize=figsize)
for s in range(n_stations):
    for c in range(n_components):
        plt.subplot(n_stations, n_components, s * n_components + c + 1)
        # define time in seconds
        time = np.linspace(moveouts[s, c], moveouts[s, c] + duration, duration) / SR
        plt.plot(time, template_waveforms[s, c, :]/np.abs(template_waveforms[s, c, :]).max(), 
                 lw=0.75, label = '{}.{}'.\
                 format(template_metadata['stations'][s].decode('utf-8'), data['metadata']['components'][c]))
        if c < 2:
            plt.axvline(time[S_wave_buffer], lw=1., ls='--', color='C3')
        else:
            plt.axvline(time[P_wave_buffer], lw=1., ls='--', color='C3')
        plt.legend(loc='best', frameon=False, handlelength=0.1)
        plt.xlim(time_min, time_max)
        if s == n_stations - 1:
            plt.xlabel('Time (s)')
        else:
            plt.xticks([])
plt.subplots_adjust(top=0.955,
        bottom=0.09,
        left=0.085,
        right=0.955,
        hspace=0.2,
        wspace=0.2)

The red dashed lines show the S-wave arrival (on the horizontal components, left two columns) and the P-wave arrival (on the vertical components, rightmost column) documented by the earthquake catalog.